# Aprendizagem Reinforçada

## Taxi Auto-Dirigível

O agente de Aprendizagem Reinforçada _(Reinforcement Learning)_ encontra um estado/cenário, e então toma uma ação de acordo com o estado/cenário atual. O objetivo é que o agente aprenda a pegar o passageiro numa posição e deixá-lo no destino.

<img src="imagens/taxienv.png" width="400px" />

- 5 x 5 = 25 possíveis posições
- Posição atual do táxi (3,1) - Linha 3 e Coluna 1
- 4 locais para pegar (pick up) e deixar (drop off) passageiros: R, G, Y B
- Locais
  - R (0,0)
  - G (0,4)
  - Y (4,0)
  - B (4,3)
- Posição do Passageiro sempre estará em AZUL
- O destino do passageiro estará sempre em ROSA
- Logo, no cenário atual o passageiro está no Y e deseja chegar no R
- As possíveis posições do passageiro são os 4 locais, mais 1 da posição de dentro do táxi
- Se a gente contabilizar todas as possíveis posições, teremos:
  - Posições do Táxi (5x5) | 5 Posições do Passageiro | 4 destinos
  - 5 x 5 x 5 x 4 = 500 estados/cenários possíveis

<table><tr>
        <td>  
        <ol>    
            <center><h4>6 Possíveis Ações</h4></center>
            <li>South (Sul)</li>
            <li>North (Norte)</li>
            <li>East  (Leste)</li>
            <li>West  (Oeste)</li>
            <li>Pickup (Pegar)</li>
            <li>Dropoff (Deixar)</li>
            </ol>
        </td>
        <td>
        <img src="imagens/rosadosventos.jpeg" width="200px"/>
        </td>    
       </tr>
</table>

<br>
<br>


In [61]:
import gym

In [62]:
env = gym.make('Taxi-v3', render_mode='ansi')
env.reset()
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




O objetivo é pegar um passageiro em um local e deixá-lo no destino requerido.

- **Recebe +20 pontos** quando deixar um passageiro no destino correto
- **Perder -1 ponto** a cada movimento que ele dá
- **Perde -10 pontos** para tentativa de pegar ou deixar o passageiro numa posição ilegal

O agente aprende a fazer 6 ações de 0-5

- 0 = south
- 1 = north
- 2 = east
- 3 = west
- 4 = pickup
- 5 = dropoff

PS1: Quando o táxi está com um passageiro dentro, sua cor muda para **verde**. <br>
PS2: Os 500 estados possíveis, numerados de 0-499 são uma codificação da posição do táxi, passageiro, e destino.
<br>PS3: O táxi não consegue fazer nenhum movimento em direção a parede. Quando o faz, **perde -1** e não sai do lugar.


In [63]:
env.reset()
print(env.render())
print('Possible actions', env.action_space)
print('Possible positions', env.observation_space)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Possible actions Discrete(6)
Possible positions Discrete(500)


| Index | Letra |
| ----- | ----- |
| 0     | R     |
| 1     | G     |
| 2     | Y     |
| 3     | B     |


### Renderizar o cenário da ilustração


In [64]:
state = env.encode(3, 1, 2, 0)
print('State', state)
env.s = state
print(env.render())

State 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




### Tabela da Recompensa

<br>{ação: [(probabilidade, próximo_estado, recompensa, terminado)]
<br>ação: [(probabilidade, próximo_estado, recompensa, terminado)]
<br>ação: [(probabilidade, próximo_estado, recompensa, terminado)]
<br>ação: [(probabilidade, próximo_estado, recompensa, terminado)]
<br>ação: [(probabilidade, próximo_estado, recompensa, terminado)]
<br>ação: [(probabilidade, próximo_estado, recompensa, terminado)]}


In [65]:
env.P[322]

{0: [(1.0, 422, -1, False)],
 1: [(1.0, 222, -1, False)],
 2: [(1.0, 342, -1, False)],
 3: [(1.0, 322, -1, False)],
 4: [(1.0, 322, -10, False)],
 5: [(1.0, 322, -10, False)]}

### Solucionando Sem Aprendizagem Reinforçada


In [83]:
"""
Mesmo após seguir o video, usar o notebook preenchido e pesquisar no google/ stack overflow 
e usar chat GPT não foi possivel achar uma solução.
"""

env.s = 322
frames = []
epochs = 0
punishments, reward = 0, 0

finished = False

while not finished:
    action = env.action_space.sample()
    state, reward, finished, info = env.step(action)

    if reward == -10:
        punishments += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward,
    })

    epochs += 1

print('Timesteps', epochs)
print('Punishments received', punishments)

ValueError: too many values to unpack (expected 4)

In [ ]:
"""
Aqui seria a visualização do que foi feito acima
"""

### Com Aprendizagem Reinforçada

#### Q-learning

Essencialmente, o agente vai aprender através das recompensas(positivas e negativas) com um tempo a tomar a melhor decisão para um determinado estado.

- Temos a tabela da recompensa P que é de onde o agente vai aprender, ao tomar uma ação no estado atual e observando a recompensa/punição, atualiza o valor-Q (Q-value).
- O valor-Q para um estado/cenário representa a "qualidade" da ação que ele irá tomar

Os valores-Q são inicializados de forma aleatória, e o agente se expõe ao ambiente, onde recebe diferentes recompensas (positivas e negativas) ao tomar diferentes ações, de forma que os valores-Q são atualizados usando a seguinte fórmula:

$$Q({\small estado}, {\small ação}) = (1 - \alpha) \cdot Q({\small estado}, {\small ação}) + \alpha \Big({\small recompensa} + {\gamma \max}_{a} Q({\small próximo \ estado}, {\small todas \ ações})\Big)$$

Onde:

- $\Large \alpha$ (Alpha) é a taxa de aprendizagem (entre 0 e 1)
- $\Large \gamma$ (Gamma) é o fator de desconto também (entre 0 e 1), que significa o quanto de importância a gente quer dar para uma recompensa. De forma que 0 faz com que o agente se preocupe apenas com a recompensa imediata. O ideal é que o agente tome as ações considerando as recompensas do estado atual, e o máximo de recompensa para o próximo estado.


# Tabela-Q

<img src="imagens/qtable.png">


- A Tabela-Q tem seus valores inicializados como 0, e depois vão sendo atualizados conforme o agente vai tomando ações no ambiente e obtendo o máximo de recompensas

### Resumindo

- Inicializa a tabela-Q com zeros
- Começa a explorar o ambiente com ações, seleciona uma de todas as ações possíveis no estado atual ($Es_{1}$)
- Vai para o próximo estado ($Es_{2}$) como resultado da ação ($A_{1}$)
- De todas as possíveis ações no estado ($Es_{2}$) seleciona a que possui o maior valor-Q
- Atualiza a tabela-Q usando a equação
- Define o próximo estado como o estado atual
- Se o objetivo é alcançado, termina, senão, repete o processo


In [84]:
import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [85]:
q_table.shape

(500, 6)

In [86]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

### Explorando valores aprendidos

Depois de explorar ações aleatórias, os valores-Q tendem a se divergirem, de forma que o agente vai poder escolher a melhor ação possível para um determinado estado.

Existe um meio termo entre explorar (escolher uma ação aleatória) e usufruir (escolher ações baseadas em valores-Q já aprendidos/treinados). Nós queremos impedir o agente de ficar toda vez fazendo os exatos movimentos, e possivelmente se super-adequando _(overfitting)_. Para evitar isso, usamos mais um parâmetro chamado $\Large \epsilon$ "epsilon" para equilibrar essas ações durante o treino do agente.

Ao invés de apenas selecionar o melhor valor-Q, algumas vezes vamos explorar novas ações. Um epsilon pequeno trás mais punições (em média), o que é natural, uma vez que estamos explorando tomando ações aleatórias.


## Treinando o Agente


In [87]:
import random

In [111]:
"""
Mesmo após seguir o video, usar o notebook preenchido e pesquisar no google/ stack overflow 
e usar chat GPT não foi possivel achar uma solução.
"""

%%time
alpha = 0.1
gamma = 0.6
epsilon = 0.1


for i in range(10000):
    state = env.reset()

    epoch, punishment, reward = 0, 0, 0
    finished = False

    while not finished:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        
        next_state, reward, finished, info = env.step(action)

        old_value = q_table[state, action]
        next_max_value = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max_value)

        q_table[state, action] = new_value

        if reward == -10:
            punishment += 1
        
        state = next_state
        epoch += 1

    clear_output(wait=True)
    print('Episodes', i + 1)
print('Finished')

UsageError: Line magic function `%%time` not found.


Agora que já treinamos o Agente, não precisamos mais explorar. Vamos apenas selecionar sempre a melhor ação escolhendo o melhor valor-Q.


In [ ]:
# Como as o conteudo abaixo depende do modelo que seria treinado acima, nao foi preenchido

## Otimizações de Hiperparâmetros


Alpha, gamma e epsilon foram definidos baseados na intuição, mas existem melhores formas de escolher os melhores parâmetros e ter um desempenho melhor.

- $\Large \alpha$ (Alpha) - (Taxa de aprendizagem) = Deveria diminuir com o tempo, para o agente aprender cada vez mais e mais
- $\Large \gamma$ (Gamma) - Quanto mais próximo você está do objetivo final, maior deveria ser a preferência para a recompensa imediata
- $\Large \epsilon$ (Epsilon) - Quanto mais experiência tem o agente, menos precisará explorar. Logo, o epsilon deve diminuir com o tempo.

- Poderíamos aplicar uma pesquisa pelos melhores parâmetros, similar ao GridSearch que vimos para os modelos preditivos.


Fontes (em inglês):

- https://gym.openai.com/
- https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/
